The aim of the code is to reduce the numebr of tangles present between the trees given as input in the newick notation and to print the optimised variables when the necessary choice is provided

In [1]:
# importing necessary packages

import re
import gurobipy as gp
from ete3 import Tree
import time



In [2]:
def Tanglegram_solver(Tree_1,Tree_2):
    start = time.time()

    # make an ete3 object using the given tree
    t1 = Tree(Tree_1)
    print("First tree : \n",t1,"\n")

    # find the nodes and their positions
    temp = re.findall(r'\d+', Tree_1)
    res_1 = list(map(int, temp))
    # print(res_1)
    positions_1 = {r:res_1.index(r) for r in res_1}
    

Newick Notation : It is a notation to represent the phylogenetic trees as a string using commas, semicolons, colons, paranthesis. Every string will end with semicolon and the paranthesis are used to group node names. Interior nodes are represented by another set of paranthesis. The sibling sub-trees come inside the same set of paranthesis.

In [ ]:
    
    # Performing same operations for tree 2
    t2 = Tree(Tree_2)
    print("Second Tree : \n",t2,"\n")
    temp = re.findall(r'\d+', Tree_2)
    res_2 = list(map(int, temp))
    positions_2 = {r:res_2.index(r) for r in res_2}

    # Number of nodes in the tree and the maximum number of variables needed during formulation (tot)
    n = len(res_1)
    tot = n*(n-1)/2

    # creating a gurobipy model and adding the varibles
    Tangle = gp.Model()
    R1 = Tangle.addMVar(int(tot),vtype = gp.GRB.BINARY,name="R1")
    R2 = Tangle.addMVar(int(tot),vtype = gp.GRB.BINARY,name="R2")
    X1 = Tangle.addMVar(Tree_1.count("("),vtype = gp.GRB.BINARY,name="X1")
    X2 = Tangle.addMVar(Tree_2.count("("),vtype = gp.GRB.BINARY,name="X2")
    C = Tangle.addMVar(int(tot),vtype = gp.GRB.BINARY,name="C")

The variables are declared using the optimvar function , specifying that the variables are integers and are either 1 or 0.
The objective function is defined as shown below.

In [ ]:
    # OBJECTIVE FUNCTION ( to be minimized )
    Tangle.setObjective(sum(C),gp.GRB.MINIMIZE)

Then the constraints are added to solve the problem.
Since a tangle is formed only when the order of nodes in the trees are different the first set of constraint relating order and tangle is given by :
C - R1 + R2 >=0
C - R2 + R1 >=0
C is the vector representing the tangles between nodes , R1,R2 - the order of the nodes in the first and second tree respectively

In [ ]:
    # adding the constraints
    Tangle.addConstr(C-R1+R2>=0,name="c1")
    Tangle.addConstr(C-R2+R1>=0,name="c2")

    # dictionaries to store the positions of the Least Common Ancestor of nodes (i,j)
    Xs1 = dict()
    Xs2 = dict()

    # function to get the position of the node configuration in the Array Variables ( R1,R2 )
    def get_num(m,n):
        return int(len(res_1)*(m-1)+n-m*(m+1)/2-1)

    global x1 , x2
    x1 , x2 = 0 , 0
    tangles = 0
    # function to find the least common ancestor of i,j and return the position of it in X1,X2
    def get_xpos(i,j):
        global x1,x2
        if (t1.get_common_ancestor(str(i),str(j)) not in Xs1.keys()):
            Xs1[t1.get_common_ancestor(str(i),str(j))] = x1
            xpos = x1
            x1 = x1+1
        else:
            xpos = Xs1[t1.get_common_ancestor(str(i),str(j))]

        if (t2.get_common_ancestor(str(i),str(j)) not in Xs2.keys()):
            Xs2[t2.get_common_ancestor(str(i),str(j))] = x2
            xpos2 = x2
            x2 = x2+1
        else:
            xpos2 = Xs2[t2.get_common_ancestor(str(i),str(j))]

        return xpos,xpos2

The second set of constraints relates subtree exchanges (X) and order ,

To start writing next set of constraints we first have to ensure that the code uses the same variable for representing the sub-tree exchange about the least common ancestor ( LCA ) for nodes with the same LCA. This is ensured using the get_xpos function which takes the node numbers as arguments and draws the subtree ( with the root as LCA of the given nodes ) and returns the variable corresponding to the least common ancestor in the X1 and X2 arrays.

The order of a set of nodes changes when a subtree exchange happens about their least common ancestor , which can be summarised as a constraint :
R = 1 - X (if the nodes are in order ( R = 1) the order becomes 0 ( out-order ) if sub-tree exchange has taken place ( X = 1 ))
R = X ( if the nodes are initially out order thhe order takes the value of X )

In [ ]:
for i in range(1,len(res_1)+1):                                 # traversing the trees and determining the constraints
        for j in range(i+1,len(res_1)+1):
            xpos , xpos2 = get_xpos(i,j)
            num = get_num(i,j)

            # if they are in order
            if positions_1[i]<positions_1[j]:
                Tangle.addConstr(R1[num]+X1[xpos] == 1)
                # print(f"R1({num+1   })+X1({xpos+1}) == 1")
                c1 = 1
            # if they are out of order
            else:
                Tangle.addConstr(R1[num]-X1[xpos] == 0)
                # print(f"R1({num+1})-X1({xpos+1}) == 0")
                c1 =0

            # similar operations for nodes of tree 2
            if positions_2[i]<positions_2[j]:
                Tangle.addConstr(R2[num] + X2[xpos2] == 1)
                # print(f"R2({num+1})+X2({xpos2+1}) == 1")
                c2 = 1
            else:
                Tangle.addConstr(R2[num] - X2[xpos2] == 0)
                # print(f"R2({num+1})-X2({xpos2+1}) == 0")
                c2 = 0

            if c1!=c2:
                tangles = tangles + 1

    after_add = time.time()
    print("\n \n Constarints are completely added at : ",after_add,"\n \n")

    # optimise
    Tangle.optimize()
    T = time.time()
    print("The time taken to solve the problem using gurobi optimiser is : ",round(T-after_add,8),"\n \n")
    swapped1 = list()
    swapped2 = list()

    for i in range(1,len(res_1)+1):
        for j in range(i+1,len(res_2)+1):
            xpos , xpos2 = get_xpos(i,j)
            if Tangle.getVarByName(f"X1[{xpos}]").x == 1 and t1.get_common_ancestor(str(i),str(j)) not in swapped1:
                t1.get_common_ancestor(str(i),str(j)).swap_children()
                swapped1.append(t1.get_common_ancestor(str(i),str(j)))


            if Tangle.getVarByName(f"X2[{xpos2}]").x == 1 and t2.get_common_ancestor(str(i),str(j)) not in swapped2:
                t2.get_common_ancestor(str(i),str(j)).swap_children()
                swapped2.append(t2.get_common_ancestor(str(i),str(j)))

Inference from the solution

In [ ]:
    print("The Exchanged Trees are : \n \n","Tree 1")
    print(t1)
    print("\n \n Tree 2 \n")
    print(t2,"\n \n")
    print("The number of Tangles are : ",C.sum().getValue() , f"Reduced from {tangles} number of tangles")

    choice = input("would you like to print the results ? (Y - yes ) : ")
    if choice == 'Y':
        # printing the solution
        for v in Tangle.getVars():
            print('%s=%g' % (v.varName, v.x))


Tanglegram_solver("((4,6),((1,5),(2,3)));","((4,(2,(1,5))),(3,6));")